<a href="https://colab.research.google.com/github/TheAgaveFairy/MusicGeneration/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/arnavmishra6996/music-generation-using-decoder-only-model-gpt2

this is the inspiration, more or less copying it but in my own typing + modifications so i understand it

In [1]:
import numpy as np
import pandas as pd
import os
import transformers
from tqdm import tqdm

from helpers import list_files

/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MIDI_TEXT_FILENAME = './inputs/midi_text_data.txt'

In [ ]:
def augment_data(midi_df):
    TRANSPOSE_RANGE = 3
    TEMPO_RANGE = 0.15
    VELOCITY_RANGE = 5

    transpose = np.random.randint(-TRANSPOSE_RANGE, TRANSPOSE_RANGE)
    tempo = np.random.uniform(1 - TEMPO_RANGE, 1 + TEMPO_RANGE)
    velocity = np.random.randint(-VELOCITY_RANGE, VELOCITY_RANGE)
    #print(f"trans: {transpose}, tempo: {tempo}")

    if np.random.random() > 0.25:
        print(f"HRT time: transing {transpose} steps")
        df['note_value'] = df['note_value'] + transpose
        
    if np.random.random() > 0.25:
        print(f"codeine time: tempo change {tempo}")
        df['start_time'] = df['start_time'] * tempo
        df['end_time'] = df['end_time'] * tempo

    if np.random.random() > 0.25:
        print(f"bad musician: velocity {velocity}")
        df['velocity'] = df['velocity'] + velocity

    return df # but isnt this already doing it in place idk honestly what best practice here is

In [ ]:
def csv_loader(path):
    """
    Returns a clean Pandas DataFrame
    I just want to do clean up the API, do type forcing etc
    """
    df = pd.read_csv(test_file_path)
    #df = df.astype({"note_value": int, "velocity": int, "instrument_program_number": int})
    for c in df:
        print(c, df[c].dtype)
    return df


In [ ]:
test_folder_path = './inputs/final/Debussy'
test_file_name = os.listdir(test_folder_path)[0]
test_file_path = os.path.join(test_folder_path, test_file_name)

#dataset = tf.data.Dataset.from_tensor_slices(os.listdir(test_folder_path))
#dataset

In [ ]:
df = csv_loader(test_file_path)
print(df.head())

augment_data(df)
print(df.head())

GPT-2 needs this as text

In [ ]:
def tokenize_midi(df, header, filename = None):
    """
    Takes in a DataFrame from a csv we made from a midi file, and turns it into GPT-2 friendly text.
    Saves it as a text file if desired and returns it as a String
    """
    
    representation = []
    representation.append(f'<|startofpiece|> {header}\n')

    for index, row in df.iterrows():
        text_line_items = []
        for name, values in row.items():
            if values.is_integer():
                text_line_items.append(f"{name}: {int(values)} ")
            else:
                text_line_items.append(f"{name}: {values} ")
            
        text_line_items[-1] = text_line_items[-1].strip()
        #text_line_items.append('\n')
        text_line = "".join(text_line_items) + "\n"
        #print(text_line)
        representation.append(text_line)
    representation.append(f'<|endofpiece|>\n')

    if filename:
        assert(filename.endswith('.txt'))
        with open(filename, 'w') as file:
            file.writelines(representation)
            
    return "".join(representation)
#a = tokenize_midi(df, "Composer: Debussy")

In [ ]:
def prepareAllData():
    root_path = './inputs/final/'
    with open(MIDI_TEXT_FILENAME, "w") as file:
        file.write("<|startoftext|>\n")
        for root, dirs, files in os.walk(root_path, topdown=False):
            composer = root.split('/')[-1]
            for filename in tqdm(files, desc=f"Processing {composer:12}"):
                temp_path = os.path.join(root, filename)
                df = pd.read_csv(temp_path)
                file.write(tokenize_midi(df, f"Composer: {composer}"))
        file.write("<|endoftext|>\n")

#prepareAllData()

Here we go

In [ ]:
"""
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            input_ids = ids[i:i+block_size]
            labels = ids[i+1:i+block_size+1] # shifted by 1
            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100

            yield {
                "input_ids": input_ids,#ids[i:i+block_size],
                "attention_mask": [1] * block_size,
                "labels": labels
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(block_size,), dtype=tf.int32)
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset(MIDI_TEXT_FILENAME, tokenizer)
"""

In [ ]:
tf.config.list_physical_devices('CPU'),tf.config.list_physical_devices('GPU')

In [ ]:
"""
#Claude 3.5Sonnet to the rescue, maybe
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling, DefaultDataCollator
from tqdm import tqdm
import os

# Initialize tokenizer and data collator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
"""
#data_collator = DataCollatorForLanguageModeling(
#    tokenizer=tokenizer,
#    mlm=True # changed from False
#)
"""
data_collator = DefaultDataCollator()

# Training configuration
class TrainingConfig:
    output_dir = "./gpt2-midi"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    save_steps = 10_000
    save_total_limit = 2
    logging_dir = './logs'
    logging_steps = 500

config = TrainingConfig()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

# Set up distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    
    # Set up the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss
    )

# Prepare the dataset
def prepare_dataset_for_training(dataset, batch_size):
    """Convert dataset to tf.data.Dataset format"""
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    """
    # Modify attention mask shape
    dataset = dataset.map(lambda x: {
        **x,
        "attention_mask": tf.reshape(x["attention_mask"], (tf.shape(x["attention_mask"])[0], 1, 1, tf.shape(x["attention_mask"])[2]))  # Reshape to (batch_size, 1, 1, seq_len)
    })
    """
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset_for_training(
    train_dataset,  # Your original train_dataset
    config.per_device_train_batch_size
)

# Custom callback for model saving
class ModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_steps, output_dir, save_total_limit):
        super().__init__()
        self.save_steps = save_steps
        self.output_dir = output_dir
        self.save_total_limit = save_total_limit
        self.step = 0

    def on_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step % self.save_steps == 0:
            # Save the model
            checkpoint_dir = os.path.join(
                self.output_dir,
                f"checkpoint-{self.step}"
            )
            self.model.save_pretrained(checkpoint_dir)

            # Remove old checkpoints if exceeding limit
            checkpoints = sorted([
                d for d in os.listdir(self.output_dir)
                if d.startswith("checkpoint-")
            ])
            while len(checkpoints) > self.save_total_limit:
                checkpoint_to_remove = os.path.join(
                    self.output_dir,
                    checkpoints.pop(0)
                )
                tf.io.gfile.rmtree(checkpoint_to_remove)

# Set up callbacks
callbacks = [
    ModelCheckpoint(
        save_steps=config.save_steps,
        output_dir=config.output_dir,
        save_total_limit=config.save_total_limit
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=config.logging_dir,
        update_freq=config.logging_steps
    )
]

# Training
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=config.num_train_epochs,
    callbacks=callbacks
)

# Save the final model and tokenizer
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)
print(f"Model saved to {config.output_dir}")
"""

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import transformers
import torch

transformers.utils.logging.set_verbosity(transformers.logging.INFO)
transformers.utils.logging.enable_progress_bar

def fine_tune_gpt2(training_file, output_dir):
    os.makedirs(output_dir, exist_ok = True)
    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Add special tokens if they're not already there
    special_tokens = {
        'additional_special_tokens': ['<|startofpiece|>', '<|endofpiece|>']
    }
    tokenizer.add_special_tokens(special_tokens)
    
    # Load model
    model = GPT2LMHeadModel.from_pretrained('gpt2', torch_dtype=torch.float16, attn_implementation="flash_attention_2")
    model.resize_token_embeddings(len(tokenizer))

    print("Model loaded")
    
    # Create dataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=training_file,
        block_size=128  # Adjust based on your data
    )

    print("Dataset loaded")
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Not using masked language modeling
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,

        logging_dir="./logs",
        logging_steps=100,
        logging_first_step=True,
        logging_strategy="steps",
        
        num_train_epochs=1,
        per_device_train_batch_size=2,
        save_steps=1000,
        #fp16=True,
        gradient_accumulation_steps=8,
        save_total_limit=1,
    )

    print("Device is", training_args.device)
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    print("Start Training")
    
    # Train
    trainer.train()    

    print("Training done, saving to", output_dir)
    
    # Save
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

In [6]:
fine_tune_gpt2(MIDI_TEXT_FILENAME, './modeloutput/')

loading file vocab.json from cache at /home/paul-dutton/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/vocab.json
loading file merges.txt from cache at /home/paul-dutton/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/paul-dutton/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json
loading file tokenizer.json from cache at /home/paul-dutton/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /home/paul-dutton/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {


Model loaded


/home/paul-dutton/miniconda3/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file ./inputs/cached_lm_GPT2Tokenizer_128_midi_text_data.txt [took 1.279 s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Dataset loaded
Device is cuda:0
Start Training


***** Running training *****
  Num examples = 460,017
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 28,751
  Number of trainable parameters = 124,441,344


ValueError: Attempting to unscale FP16 gradients.

In [ ]:
def generate_music(model_path, prompt="Composer: Bach", max_length=512, num_return_sequences=1, temperature=1.0):
    """
    Generate music using your fine-tuned GPT-2 model
    
    Args:
        model_path: Path to the fine-tuned model
        prompt: Text prompt to start generation (e.g., "Composer: Bach")
        max_length: Maximum length of generated sequence
        num_return_sequences: Number of different sequences to generate
        temperature: Controls randomness (lower = more deterministic)
        
    Returns:
        List of generated MIDI objects
    """
    # Load model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    
    # Format the prompt with the right structure
    formatted_prompt = f"<|startofpiece|>\n{prompt}\n"
    
    # Encode the prompt
    input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt")
    
    # Generate continuation
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )
    
    # Decode and parse the output
    generated_sequences = []
    for sequence in output_sequences:
        text = tokenizer.decode(sequence, skip_special_tokens=False)
        
        # Make sure we stop at the end of piece token if present
        if "<|endofpiece|>" in text:
            text = text.split("<|endofpiece|>")[0] + "<|endofpiece|>"
            
        generated_sequences.append(text)
        
    # Convert text to MIDI
    midi_objects = [text_to_midi(seq) for seq in generated_sequences]
    
    return midi_objects, generated_sequences

midi_objects, generated_sequences = generate_music